# Stock Tracker - Percentage Change Analysis - Compiled


📌 The Four Pick Lists

Gemini Picks
1.	SNPS – Synopsys
2.	TMO – Thermo Fisher
3.	ADBE – Adobe
4.	ORCL – Oracle
5.	LLY – Eli Lilly

Grok Picks
1.	TSLA – Tesla
2.	AVGO – Broadcom
3.	GOOGL – Alphabet
4.	ADBE – Adobe
5.	UNH – UnitedHealth

ChatGPT Round 1 (Markdown Scanner)
1.	NVDA – Nvidia
2.	AVGO – Broadcom
3.	AAPL – Apple
4.	ORCL – Oracle
5.	TSLA – Tesla

ChatGPT Round 2 (SQL scanner)
1.	AAPL – Apple
2.	ORCL – Oracle
3.	ADBE – Adobe
4.	HOOD – Robinhood
5.	TSLA – Tesla

⸻

✅ Overlaps Across Models
• ADBE (Adobe) → Gemini, Grok, ChatGPT Round 2
• ORCL (Oracle) → Gemini, ChatGPT Round 1 & 2
• TSLA (Tesla) → Grok, ChatGPT Round 1 & 2
• AVGO (Broadcom) → Grok, ChatGPT Round 1
• AAPL (Apple) → ChatGPT Round 1 & 2

⸻

📊 Consensus Ranking (by frequency of appearance)

3 models picked:
• ADBE (Gemini, Grok, ChatGPT 2)
• ORCL (Gemini, GPT1, GPT2)
• TSLA (Grok, GPT1, GPT2)

2 models picked:
• AVGO (Grok, GPT1)
• AAPL (GPT1, GPT2)

1 model picked (unique/special mentions):
• NVDA (GPT1 only)
• SNPS (Gemini only)
• TMO (Gemini only)
• LLY (Gemini only)
• GOOGL (Grok only)
• UNH (Grok only)
• HOOD (GPT2 only)

⸻

🏆 Core Consensus Basket (appearing in ≥3 models)
• ADBE – Adobe
• ORCL – Oracle
• TSLA – Tesla

⸻

🔮 Expanded Consensus Top 5 (add strongest 2 from 2-model overlap)
1. ADBE – Adobe (3 models)
2. ORCL – Oracle (3 models)
3. TSLA – Tesla (3 models)
4. AVGO – Broadcom (2 models)
5. AAPL – Apple (2 models)


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import yfinance as yf
import time  # For retries
import logging  # Optional: For better logging

# Set up logging (replace print with logging for production)
logging.basicConfig(level=logging.INFO)

print("Libraries imported successfully!")  # Keeping print for consistency with original
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Using Plotly for interactive visualizations! 📊")

# Stock Tickers Configuration
STOCK_TICKERS = []

# Use realistic past dates for actual data
BASELINE_DATE = '2000-01-01'  # Changed to past date that exists

# Historical Comparison - Compare performance to current date
future_date_str = datetime.now().strftime('%Y-%m-%d')  # Use current date for comparison

# Will store baseline prices after fetching

baseline_prices = {}

print("✅ Stock tickers configured successfully!")
print(f"📅 Baseline date: {BASELINE_DATE}")
print(f"📈 Tracking {len(STOCK_TICKERS)} stocks")

class StockTracker:
    def __init__(self, tickers: list[str], baseline_date: str):
        self.tickers = tickers
        self.baseline_date = baseline_date
        self.baseline_prices = self.fetch_stock_prices(baseline_date)

    def fetch_stock_prices(self, date_str: str | None = None, compare_to_baseline: bool = False) -> dict[str, float]:
        """
        Fetch current or historical prices for all stocks in self.tickers using batch download
        
        Parameters:
        - date_str: Optional date string in 'YYYY-MM-DD' format. If None, gets latest prices.
        - compare_to_baseline: If True, shows percentage change from baseline
        
        Returns:
        - dict: {ticker: price}
        """
        if date_str is None:
            date_str = datetime.now().strftime('%Y-%m-%d')
            print(f"📈 Fetching LATEST stock prices (around {date_str})")
        else:
            print(f"📈 Fetching stock prices for: {date_str}")
        
        print("=" * 60)
        
        fetched_prices = {}
        try:
            start_date = datetime.strptime(date_str, '%Y-%m-%d') - timedelta(days=7)  # Buffer for non-trading days
            end_date = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)
            
            for attempt in range(3):  # Retry up to 3 times
                data = yf.download(self.tickers, start=start_date, end=end_date)['Close']
                if not data.empty:
                    break
                print(f"Retry {attempt + 1}/3: No data, waiting...")
                time.sleep(2)
            
            if data.empty:
                print("❌ No data available for the specified period.")
                return {}
            
            successful_fetches = 0
            for ticker in self.tickers:
                if ticker in data.columns and not data[ticker].dropna().empty:
                    price = round(data[ticker].dropna().iloc[-1], 2)  # Latest available close
                    fetched_prices[ticker] = price
                    if compare_to_baseline and self.baseline_prices and ticker in self.baseline_prices:
                        baseline_price = self.baseline_prices[ticker]
                        if baseline_price is not None:
                            pct_change = ((price - baseline_price) / baseline_price) * 100
                            print(f"🔍 {ticker}: ${price:.2f} ({pct_change:+.1f}%)")
                        else:
                            print(f"🔍 {ticker}: ${price:.2f} (No baseline)")
                    else:
                        print(f"🔍 {ticker}: ${price:.2f}")
                    successful_fetches += 1
                else:
                    print(f"🔍 {ticker}: Failed")
            
            print(f"\n✅ Successfully fetched prices for {successful_fetches}/{len(self.tickers)} stocks")
            return fetched_prices
        except Exception as e:
            print(f"❌ Error in batch fetch: {e}")
            return {}

    def track_stocks(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Track stock performance from baseline to target date
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        results = []
        
        print(f"📈 STOCK PERFORMANCE ANALYSIS")
        print(f"From: {self.baseline_date} → To: {target_date}")
        print("=" * 50)
        
        for symbol, current_price in current_prices.items():
            if symbol in self.baseline_prices and current_price is not None:
                baseline_price = self.baseline_prices[symbol]
                if baseline_price is not None:
                    price_change = current_price - baseline_price
                    pct_change = (price_change / baseline_price) * 100
                    
                    results.append({
                        'Symbol': symbol,
                        'Baseline': baseline_price,
                        'Current': current_price,
                        'Change': price_change,
                        'Percent_Change': pct_change
                    })
        
        if not results:
            print("No valid price data provided.")
            return None
        
        # Create DataFrame and sort by performance
        df = pd.DataFrame(results)
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Display results
        print(f"{'Rank':<4} {'Symbol':<6} {'Baseline':<10} {'Current':<10} {'Change':<10} {'% Change':<10}")
        print("-" * 60)
        
        for i, (_, row) in enumerate(df.iterrows(), 1):
            print(f"{i:<4} {row['Symbol']:<6} ${row['Baseline']:<9.2f} ${row['Current']:<9.2f} "
                  f"${row['Change']:<9.2f} {row['Percent_Change']:<9.2f}%")
        
        # Summary
        avg_change = df['Percent_Change'].mean()
        print(f"\n🏆 Best Performer: {df.iloc[0]['Symbol']} ({df.iloc[0]['Percent_Change']:+.2f}%)")
        print(f"📉 Worst Performer: {df.iloc[-1]['Symbol']} ({df.iloc[-1]['Percent_Change']:+.2f}%)")
        print(f"📊 Average Change: {avg_change:+.2f}%")
        
        return df

    def create_performance_plot(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Create interactive visualization for stock performance using Plotly
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        df = self.track_stocks(target_date, current_prices)
        
        if df is None:
            return None
        
        # Sort x-axis by performance
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Calculate average change
        avg_change = df['Percent_Change'].mean()
        
        # Create color scheme based on performance
        colors = ['#00CC44' if x > 0 else '#FF4444' for x in df['Percent_Change']]
        
        # Create interactive bar chart
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=df['Symbol'],
            y=df['Percent_Change'],
            marker_color=colors,
            text=[f'{x:.1f}%' for x in df['Percent_Change']],
            textposition='outside',
            hovertemplate=
            '<b>%{x}</b><br>' +
            'Baseline: $%{customdata[0]:.2f}<br>' +
            'Current: $%{customdata[1]:.2f}<br>' +
            'Change: %{y:.2f}%<br>' +
            '<extra></extra>',
            customdata=list(zip(df['Baseline'], df['Current'])),
            showlegend=False
        ))
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
        
        # Add average change line
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg Change: {avg_change:.2f}%",
            annotation_position="bottom right"
        )
        
        # Add annotations for best and worst
        fig.add_annotation(x=df['Symbol'].iloc[0], y=df['Percent_Change'].iloc[0] + 5,
                           text="Best", showarrow=True, arrowhead=1)
        fig.add_annotation(x=df['Symbol'].iloc[-1], y=df['Percent_Change'].iloc[-1] - 5,
                           text="Worst", showarrow=True, arrowhead=1)
        
        # Update layout
        fig.update_layout(
            title={
                'text': f'📈 Stock Performance: {self.baseline_date} → {target_date}',
                'x': 0.5,
                'font': {'size': 20, 'family': 'Arial Black'}
            },
            xaxis_title="Stock Symbol",
            yaxis_title="Percentage Change (%)",
            height=600,
            width=1000,
            template="plotly_white",
            font=dict(size=12),
            hovermode='x',
            plot_bgcolor='rgba(240, 240, 240, 0.1)'
        )
        
        # Update axes
        fig.update_xaxes(tickangle=45, tickfont=dict(size=11), categoryorder='array', categoryarray=df['Symbol'].tolist())
        fig.update_yaxes(tickfont=dict(size=11), gridcolor='rgba(128, 128, 128, 0.3)')
        
        # Show the interactive plot
        fig.show()
        
        # Optional: Export
        # fig.write_html('performance.html')
        # fig.write_image('performance.png')
        
        return df


def run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str):
    """
    Run complete stock trading analysis from baseline to target date
    
    Parameters:
    - STOCK_TICKERS: List of stock ticker symbols
    - BASELINE_DATE: Baseline date string (YYYY-MM-DD)
    - future_date_str: Target date string (YYYY-MM-DD)
    
    Returns:
    - pd.DataFrame: Performance analysis results
    """
    # Initialize tracker
    tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

    print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
    print(tracker.baseline_prices)

    print("✅ Stock tracking class ready!")
    print("📊 Interactive Plotly visualization ready!")

    print(f"📊 HISTORICAL COMPARISON")
    print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
    print("=" * 50)

    # Fetch prices for future date
    future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

    # DEMO: Interactive Plotly Visualizations
    print("\n🎯 Historical Performance Analysis")
    print("=" * 45)

    print("Creating interactive Plotly charts...")
    print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
    demo_result = tracker.create_performance_plot(future_date_str, future_prices)

    return demo_result

Libraries imported successfully!
Date: 2025-09-16 22:27:03
Using Plotly for interactive visualizations! 📊
✅ Stock tickers configured successfully!
📅 Baseline date: 2000-01-01
📈 Tracking 0 stocks


In [2]:
# Use realistic past dates for actual data
BASELINE_DATE = '2025-09-07' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-16'

In [3]:
# Stock Tickers Configuration (Consensus Top 5)
STOCK_TICKERS = [
    'ADBE',  # Adobe Inc.
    'ORCL',  # Oracle Corp.
    'TSLA',  # Tesla Inc.
    'BMNR',  # Bitmine
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

📈 Fetching stock prices for: 2025-09-07
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  4 of 4 completed


🔍 ADBE: $348.97
🔍 ORCL: $232.80
🔍 TSLA: $350.84
🔍 BMNR: $42.04

✅ Successfully fetched prices for 4/4 stocks

📊 Baseline Prices (2025-09-07):
{'ADBE': 348.97, 'ORCL': 232.8, 'TSLA': 350.84, 'BMNR': 42.04}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-07 → Target: 2025-09-16
📈 Fetching stock prices for: 2025-09-16


[*********************100%***********************]  4 of 4 completed


🔍 ADBE: $350.80 (+0.5%)
🔍 ORCL: $308.52 (+32.5%)
🔍 TSLA: $416.61 (+18.7%)
🔍 BMNR: $51.02 (+21.4%)

✅ Successfully fetched prices for 4/4 stocks

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-07 to 2025-09-16:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-07 → To: 2025-09-16
Rank Symbol Baseline   Current    Change     % Change  
------------------------------------------------------------
1    ORCL   $232.80    $308.52    $75.72     32.53    %
2    BMNR   $42.04     $51.02     $8.98      21.36    %
3    TSLA   $350.84    $416.61    $65.77     18.75    %
4    ADBE   $348.97    $350.80    $1.83      0.52     %

🏆 Best Performer: ORCL (+32.53%)
📉 Worst Performer: ADBE (+0.52%)
📊 Average Change: +18.29%


In [4]:
# Stock Tickers Configuration (Consensus Top 5)
STOCK_TICKERS = [
    'ADBE',  # Adobe Inc.
    'ORCL',  # Oracle Corp.
    'TSLA',  # Tesla Inc.
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

[*********************100%***********************]  3 of 3 completed

📈 Fetching stock prices for: 2025-09-07
🔍 ADBE: $348.97
🔍 ORCL: $232.80
🔍 TSLA: $350.84

✅ Successfully fetched prices for 3/3 stocks

📊 Baseline Prices (2025-09-07):
{'ADBE': 348.97, 'ORCL': 232.8, 'TSLA': 350.84}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-07 → Target: 2025-09-16
📈 Fetching stock prices for: 2025-09-16



[*********************100%***********************]  3 of 3 completed

🔍 ADBE: $350.80 (+0.5%)
🔍 ORCL: $308.36 (+32.5%)
🔍 TSLA: $416.53 (+18.7%)

✅ Successfully fetched prices for 3/3 stocks

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-07 to 2025-09-16:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-07 → To: 2025-09-16
Rank Symbol Baseline   Current    Change     % Change  
------------------------------------------------------------
1    ORCL   $232.80    $308.36    $75.56     32.46    %
2    TSLA   $350.84    $416.53    $65.69     18.72    %
3    ADBE   $348.97    $350.80    $1.83      0.52     %

🏆 Best Performer: ORCL (+32.46%)
📉 Worst Performer: ADBE (+0.52%)
📊 Average Change: +17.24%


In [5]:
# Stock Tickers Configuration (Consensus Top 5)
STOCK_TICKERS = [
    'ADBE',  # Adobe Inc.
    'ORCL',  # Oracle Corp.
    'AVGO',  # Broadcom Inc.
    'TSLA',  # Tesla Inc.
    'NVDA',  # Nvidia Corp.
    'AAPL',  # Apple Inc.
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

[****************      33%                       ]  2 of 6 completed

📈 Fetching stock prices for: 2025-09-07


[*********************100%***********************]  6 of 6 completed


🔍 ADBE: $348.97
🔍 ORCL: $232.80
🔍 AVGO: $334.89
🔍 TSLA: $350.84
🔍 NVDA: $167.01
🔍 AAPL: $239.69

✅ Successfully fetched prices for 6/6 stocks

📊 Baseline Prices (2025-09-07):
{'ADBE': 348.97, 'ORCL': 232.8, 'AVGO': 334.89, 'TSLA': 350.84, 'NVDA': 167.01, 'AAPL': 239.69}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-07 → Target: 2025-09-16
📈 Fetching stock prices for: 2025-09-16


[*********************100%***********************]  6 of 6 completed

🔍 ADBE: $350.84 (+0.5%)
🔍 ORCL: $308.34 (+32.4%)
🔍 AVGO: $360.29 (+7.6%)
🔍 TSLA: $416.94 (+18.8%)
🔍 NVDA: $176.43 (+5.6%)
🔍 AAPL: $239.66 (-0.0%)

✅ Successfully fetched prices for 6/6 stocks

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-07 to 2025-09-16:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-07 → To: 2025-09-16
Rank Symbol Baseline   Current    Change     % Change  
------------------------------------------------------------
1    ORCL   $232.80    $308.34    $75.54     32.45    %
2    TSLA   $350.84    $416.94    $66.10     18.84    %
3    AVGO   $334.89    $360.29    $25.40     7.58     %
4    NVDA   $167.01    $176.43    $9.42      5.64     %
5    ADBE   $348.97    $350.84    $1.87      0.54     %
6    AAPL   $239.69    $239.66    $-0.03     -0.01    %

🏆 Best Performer: ORCL (+32.45%)
📉 Worst Performer: AAPL (-0.01%)
📊 Average Change: +10.84%


In [6]:
# Stock Tickers Configuration (Full Consensus + Unique Combined)
STOCK_TICKERS = [
    # Core Consensus (3 models)
    'ADBE',  # Adobe Inc.
    'ORCL',  # Oracle Corporation
    'TSLA',  # Tesla Inc.

    # Strong Overlap (2 models)
    'AVGO',  # Broadcom Inc.
    'AAPL',  # Apple Inc.

    # Unique Mentions (1 model each)
    'NVDA',  # Nvidia Corp.
    'SNPS',  # Synopsys, Inc.
    'TMO',   # Thermo Fisher Scientific
    'LLY',   # Eli Lilly & Co.
    'GOOGL', # Alphabet Inc.
    'UNH',   # UnitedHealth Group Inc.
    'HOOD',  # Robinhood Markets Inc.
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

[****************      33%                       ]  4 of 12 completed

📈 Fetching stock prices for: 2025-09-07


[*********************100%***********************]  12 of 12 completed


🔍 ADBE: $348.97
🔍 ORCL: $232.80
🔍 TSLA: $350.84
🔍 AVGO: $334.89
🔍 AAPL: $239.69
🔍 NVDA: $167.01
🔍 SNPS: $598.14
🔍 TMO: $492.16
🔍 LLY: $727.21
🔍 GOOGL: $234.79
🔍 UNH: $315.39
🔍 HOOD: $101.25

✅ Successfully fetched prices for 12/12 stocks

📊 Baseline Prices (2025-09-07):
{'ADBE': 348.97, 'ORCL': 232.8, 'TSLA': 350.84, 'AVGO': 334.89, 'AAPL': 239.69, 'NVDA': 167.01, 'SNPS': 598.14, 'TMO': 492.16, 'LLY': 727.21, 'GOOGL': 234.79, 'UNH': 315.39, 'HOOD': 101.25}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-07 → Target: 2025-09-16
📈 Fetching stock prices for: 2025-09-16


[*********************100%***********************]  12 of 12 completed

🔍 ADBE: $350.35 (+0.4%)
🔍 ORCL: $308.21 (+32.4%)
🔍 TSLA: $416.82 (+18.8%)
🔍 AVGO: $360.38 (+7.6%)
🔍 AAPL: $239.60 (-0.0%)
🔍 NVDA: $176.43 (+5.6%)
🔍 SNPS: $424.76 (-29.0%)
🔍 TMO: $475.45 (-3.4%)
🔍 LLY: $753.55 (+3.6%)
🔍 GOOGL: $250.64 (+6.8%)
🔍 UNH: $341.14 (+8.2%)
🔍 HOOD: $114.86 (+13.4%)

✅ Successfully fetched prices for 12/12 stocks

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-07 to 2025-09-16:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-07 → To: 2025-09-16
Rank Symbol Baseline   Current    Change     % Change  
------------------------------------------------------------
1    ORCL   $232.80    $308.21    $75.41     32.39    %
2    TSLA   $350.84    $416.82    $65.98     18.81    %
3    HOOD   $101.25    $114.86    $13.61     13.44    %
4    UNH    $315.39    $341.14    $25.75     8.16     %
5    AVGO   $334.89    $360.38    $25.49     7.61     %
6    GOOGL  $234.79    $250.64    $15.85     6.75     %
7    NVDA   $167.01    $176.4